In [2]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
#from collections import namedtuple

# Get data (file : valeursfoncieres-2020 )

In [5]:
def get_data(url):
  df = pd.read_csv(url, low_memory=False)
  print("data.shape : ", df.shape)
  return df
  
#get_data form url 
url = 'https://files.data.gouv.fr/geo-dvf/latest/csv/2020/full.csv.gz'
df = get_data(url)
# make copies
whole_data = df.copy()

def drop_rows_with_nan_in_column(df, col_name): #df : dataframe
   selected_rows = df[df[col_name].notna()]
   return selected_rows

data_clean = drop_rows_with_nan_in_column(df = whole_data, col_name= 'valeur_fonciere')
print("data_clean", data_clean.shape, "whole_data", whole_data.shape)


def filtered_data(df, col_mutation, col_local_type):
  
  modif_df = df[
                (df[col_mutation] == "Vente") & \
                 (   \
                  (df[col_local_type] == "Maison") | 
                   (df[col_local_type] == "Appartement") | 
                  (df[col_local_type] == "Dépendance")
                 )
                 ] 

  return modif_df                                
 
data = filtered_data(df = data_clean, 
                           col_mutation = 'nature_mutation',
                           col_local_type = 'type_local')                          

print("data.shape (whole)", whole_data.shape) # whole data
print("data_clean.shape", data_clean.shape) # NaN 'valeur_fonciere' out

print("data.shape (industrial out, NaN local type out, and only 'vente'mutation ", data.shape)

data.shape :  (2459560, 40)
data_clean (2427808, 40) whole_data (2459560, 40)
data.shape (whole) (2459560, 40)
data_clean.shape (2427808, 40)
data.shape (industrial out, NaN local type out, and only 'vente'mutation  (1203720, 40)


In [7]:
data['code_postal'].isnull().sum()

37

## Select rows with NaN in column

In [8]:
selected_rows = data[data['code_postal'].isnull()]
selected_rows.head(2)

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,ancien_code_commune,ancien_nom_commune,id_parcelle,ancien_id_parcelle,numero_volume,lot1_numero,lot1_surface_carrez,lot2_numero,lot2_surface_carrez,lot3_numero,lot3_surface_carrez,lot4_numero,lot4_surface_carrez,lot5_numero,lot5_surface_carrez,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
81601,2020-32124,2020-05-26,1,Vente,104300.0,2.0,.,RUE ANTONIN CORONAT,0072,NaN,05061,Gap,05,NaN,NaN,05061000CY0138,NaN,NaN,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,6.071288,44.566967
81602,2020-32124,2020-05-26,1,Vente,104300.0,2.0,.,RUE ANTONIN CORONAT,0072,NaN,05061,Gap,05,NaN,NaN,05061000CY0138,NaN,NaN,22,NaN,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2.0,Appartement,30.0,1.0,NaN,NaN,NaN,NaN,NaN,6.071288,44.566967


In [9]:
selected_rows.shape

(37, 40)

In [10]:
def select_rows_with_nan_in_column(df, col_name): #df : dataframe
  selected_rows = data[data[col_name].isnull()]
  return selected_rows
df_test = select_rows_with_nan_in_column(df = data, col_name= 'code_postal')
df_test.shape

(37, 40)

In [11]:

selected_rows['valeur_fonciere'].isnull().sum()

0

In [12]:
mask_columns = ['adresse_numero', 'adresse_suffixe',
       'adresse_nom_voie', 
        'code_postal', 'code_commune',
       'code_departement',
       'valeur_fonciere',  'type_local',
        'code_postal', 'id_parcelle']

selected_rows[mask_columns].head(5)

,adresse_numero,adresse_suffixe,adresse_nom_voie,code_postal,code_commune,code_departement,valeur_fonciere,type_local,code_postal,id_parcelle
81601,2.0,.,RUE ANTONIN CORONAT,NaN,05061,05,104300.0,Dépendance,NaN,05061000CY0138
81602,2.0,.,RUE ANTONIN CORONAT,NaN,05061,05,104300.0,Appartement,NaN,05061000CY0138
139884,739.0,NaN,AV JEAN MARCHAND,NaN,06161,06,114000.0,Appartement,NaN,06161000AT0023
139969,31.0,NaN,AV JEAN MARCHAND,NaN,06161,06,4500.0,Dépendance,NaN,06161000AT0003
140008,629.0,NaN,AV JEAN MARCHAND,NaN,06161,06,248000.0,Appartement,NaN,06161000AT0007


In [13]:
selected_rows['adresse_suffixe'].unique() 

array(['.', nan, 'F'], dtype=object)

In [14]:
selected_rows['adresse_suffixe'].value_counts()

.    5
F    1
Name: adresse_suffixe, dtype: int64

In [15]:
selected_rows['adresse_suffixe'].isnull().sum() # There are 31 missing values  (31/37)

31

In [16]:
def get_count_of_missing_values(df):
  missing_df = pd.DataFrame(df.isnull().sum().sort_values(ascending=False))
  return missing_df

missing_df = get_count_of_missing_values(df = selected_rows)
#missing_df

In [17]:
mask_columns = ['adresse_numero', 'adresse_suffixe',
       'adresse_nom_voie', 
        'code_postal', 'code_commune',
       'code_departement',
        'code_postal', 'id_parcelle']


In [18]:
(missing_df.T)[mask_columns]

,adresse_numero,adresse_suffixe,adresse_nom_voie,code_postal,code_commune,code_departement,code_postal,id_parcelle
0,0,31,0,37,0,0,37,0


**adresse_numero & adresse_nom_voie & code_commune & code_departement**: all  values exist

**adresse_suffixe** : 31 missing values  (31/37)

**adresse_suffixe** : 5 values (5/6) are labeled by .

----

.    5

----

F    1

If there are **Missing values** in column **'code_postale'**, we look at the column **'adresse_suffixe'**.
If in column **'adresse_suffixe'**,  there are a lot of missing values and/or '.'value, we drop rows of the column **'code_postale'**, with NaN